In [ ]:
!pip install konlpy
!pip install gensim

In [6]:
import pandas as pd
import numpy as np
import os
import sys


from  util import load_data
from  util.text_preprocessing import  tokenize_okt, count_comment

import tqdm

In [5]:
# Load Data

ImportError: cannot import name 'load_data' from 'util.text_preprocessing' (C:\Users\multicampus\PycharmProjects\s03p23a201\backend\ML\comment_6emo\news-comments_emotion_classification-master\util\text_preprocessing.py)

# Load Data

In [7]:
DATA_PATH = os.path.normpath('dataset/daum_comments_2017/')

In [8]:
[path for path in os.listdir(os.path.join(DATA_PATH,"raw_data")) if (path[-3:] == 'txt')&('non' in path)]

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'dataset\\daum_comments_2017\\raw_data'

In [7]:
total_comments_df = pd.DataFrame()
path_only_txt = [path for path in os.listdir(os.path.join(DATA_PATH,"raw_data")) \
                 if (path[-3:] == 'txt')&('non' in path)]

for path in path_only_txt:
    print(path)
    data_df = load_data(os.path.join(DATA_PATH,'raw_data',path))
    data_df = data_df[data_df.comment != ""]
    total_comments_df = total_comments_df.append(data_df)

daum_comments_201712_non_entertain.txt
312267
@@@@@@
daum_comments_201709_non_entertain.txt
433686
@@@@@@
daum_comments_201710_non_entertain.txt
34022
@@@@@@
daum_comments_201708_non_entertain.txt
411888
@@@@@@
daum_comments_201711_non_entertain.txt
220887
@@@@@@
daum_comments_201707_non_entertain.txt
371725
@@@@@@


In [6]:
#total_comments_df.to_pickle(os.path.join(DATA_PATH,"200710 non_entertain-total-comments.pkl"))
total_comments_df = pd.read_pickle(os.path.join(DATA_PATH,"200710 non_entertain-total-comments.pkl"))
print(total_comments_df.shape)
total_comments_df.sample(5)

(1780402, 3)


,news_id,comment_id,comment
57471,20171107054548817,4404D996071D4E12857BE97F5ED5B9F4,억울해서가 아니라 불안해서 겟지...
88322,20170809092231879,BA95196182774614827AADCBA0551FD3,어린것들이 일찍 결혼하면 이런 문제가 발생 할다니깐.....................
154714,20171211063031702,331259E8A1A541C2BB23C4940543F158,왜 학부모가 하는건데? 고학년학생 선생님들이 나와서해라..
39479,20170703114432116,E4B7B213FB5E4F1B96CAC8F38786CC8F,신고포상금 꼭 드리세요
215829,20170818165739229,C83A7F51E15E42DCB256DC4A2A390240,나이에 상관없이 혼자 사는 사람들은 정기적으로 동사무소 같은 곳에서 생사 확인했으면


# Tokenize comments

- 추출 태그 = [명사 동사 형용사]
- stopwords = ["것", "이", "안", "더", "왜", "때", "좀", "뭐", "거", "저", "뿐", "머"]

In [7]:
token_data = []

for comment in tqdm.tqdm(total_comments_df.comment):
    token = tokenize_okt(comment)
    token_data.append(token)                                                                                           

100%|██████████| 1780402/1780402 [2:03:15<00:00, 240.75it/s]  


In [14]:
keys, n_vocab = count_comment(token_data)

있다(266291) 되다(227684) 없다(198500) 아니다(179739) 보다(158555) 사람(137378) 같다(109397) 들다(105248) 않다(104030) 그렇다(99124) 가다(93559) 놈(93065) 국민(90584) 받다(88033) 말(86876) 먹다(80270) 돈(77413) 나라(76930) 좋다(72559) 왜(72217) 한(70275) 더(70250) 때(67161) 생각(66934) 시키다(65723) 뭐(64864) 이렇다(64203) 개(63970) 많다(61419) 좀(59965) 알다(57425) 우리(55295) 거(54994) 인간(53780) 한국(53405) 만들다(52883) 내(52726) 나오다(51075) 살다(50656) 모르다(49423) 일(49036) 주다(48805) 지다(48591) 기사(48204) 안되다(47479) 니(47393) 문제(47363) 차다(45889) 대통령(45612) 중국(44103) 
Total Vocab:  168620



In [15]:
series_token_data = pd.Series(token_data)
series_token_data[:5]

0    [이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없다, 물, 심양,...
1          [문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하다, 비다]
2                    [단, 한번, 써다, 보지, 포인트, 절반, 통신비, 차감]
3                        [우리나라, 통신, 업체, 차다, 쉬다, 돈, 벌다]
4                                     [진짜, 쓸다, 너무, 적다]
dtype: object

In [16]:
total_comments_df = total_comments_df.reset_index(drop=True)
total_comments_df['comment_token'] = series_token_data
total_comments_df.tail()

,news_id,comment_id,comment,comment_token
1780397,20170731044235632,6D6963D736FF40328D170B475A4F34F0,인천에 자유 뭐시기 소속 구청장이 좀 있는데 어디냐 서구냐 남구냐 연수구냐 남동구냐...,"[인천, 자유, 뭐시기, 소속, 구청, 좀, 있다, 어디, 서구, 남구, 연수구, ..."
1780398,20170731044235632,0FB426A0CFF2443D92E8D96402008692,맹바기닮은것보니 자유당일듯...,"[맹바기, 닮다, 보다, 자유당]"
1780399,20170731044235632,897F2E5AC2D843628B24244DF4FF77BB,멍박그네 정권 싸 똥치우는 문정부 개고생. 적폐청산 MB 방산 자원외교 4대강 ...,"[멍박, 그네, 정권, 싸다, 똥, 치우다, 정부, 개, 고생, 적폐, 청산, 방산..."
1780400,20170731044235632,80A31650BF054D2AAE92310374373DB9,이제 명박이좀 잊을때도 안됐나 그만하고 새출발하자 듣는것도 지겹다,"[이제, 명박, 좀, 잊다, 때, 돼다, 그만하다, 출발, 하자, 듣다, 지겹다]"
1780401,20170731044235632,B9643D95F2F045A9808C2CAA0B70F25B,C구청장을 당장 구속 수사하면 될듯~~~,"[청장, 당장, 구속, 수사, 되다]"


In [17]:
total_comments_df.to_pickle("dataset/total_raw_token/0710 comments_with_okt-tokenized.pkl")

In [18]:
token_data = pd.read_pickle("dataset/total_raw_token/0710 comments_with_okt-tokenized.pkl")

In [19]:
token_data.head()

,news_id,comment_id,comment,comment_token
0,20171201145847479,F3C2DFCFE9E947B19C8A3053DD5C821B,이국종교수는 외과 야전사령관 이다. 그분의 업무에 차질 없도록 물심양면 으로 ...,"[이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없다, 물, 심양,..."
1,20171201145847479,61A3B6AAD6124DFFB70EC4275496F8A6,"문재인 대통령님과 이국종 교수님, 그리고 귀순병사 모두 건강하시길 빕니다.","[문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하다, 비다]"
2,20171201060244786,B6660EF2C7C14A2F903A08363641CF09,단 한번도 써보지 못했다ㆍ 포인트의 절반만이라도 통신비로 차감을!,"[단, 한번, 써다, 보지, 포인트, 절반, 통신비, 차감]"
3,20171201060244786,C4F129BFB1FD4543A24A03D96F38351D,우리나라 통신업체들 참 쉽게 돈 벌어요,"[우리나라, 통신, 업체, 차다, 쉬다, 돈, 벌다]"
4,20171201060244786,1A21FDDF151E45DF808F0E5B662E8F3C,진짜 쓸데가너무적다.,"[진짜, 쓸다, 너무, 적다]"


In [20]:
tokens = token_data['comment_token']
tokens[:5]

0    [이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없다, 물, 심양,...
1          [문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하다, 비다]
2                    [단, 한번, 써다, 보지, 포인트, 절반, 통신비, 차감]
3                        [우리나라, 통신, 업체, 차다, 쉬다, 돈, 벌다]
4                                     [진짜, 쓸다, 너무, 적다]
Name: comment_token, dtype: object

# Word Embedding 

두 모델 학습해서 이후 결과 비교해볼 것 
 - tokenized_comments
     - word2vec
     - fastText
 - 자모 단위 분해 + fastText
 
**Reference**    
- [Finding best fasttext hyperparameters](http://soner.in/fasttext-grid-search/)
- [한국어 단어 임베딩을 위한 Word2vec 모델의 최적화](http://journal.dcs.or.kr/xml/19540/19540.pdf)
- [FastText, Word representation using subword](https://lovit.github.io/nlp/representation/2018/10/22/fasttext_subword/)

In [21]:
import codecs
from gensim.models import word2vec
from gensim.models import fasttext

## word2vec

In [22]:
import multiprocessing
config = {
    'min_count': 1,  # 등장 횟수가 1 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 20,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
    'window': 5,
    'seed': 25 #random number,
}

In [23]:
# 모델 학습
model_w2v = word2vec.Word2Vec(tokens,**config)

In [25]:
#모델 저장
model_w2v.save(os.path.join("dataset/model/0710 w2v_model"))

In [26]:
# 모델 불러오기
w2v_model = word2vec.Word2Vec.load("dataset/model/0710 w2v_model")

### analogy test

In [16]:
w2v_model.wv.most_similar(positive = "중국")

[('짱깨', 0.7469527721405029),
 ('짱개', 0.7402806878089905),
 ('중공', 0.7356770038604736),
 ('짱꼴라', 0.6442628502845764),
 ('러시아', 0.6201988458633423),
 ('중국인', 0.6164897084236145),
 ('중궈', 0.5840282440185547),
 ('시진핑', 0.5833264589309692),
 ('일본', 0.560553789138794),
 ('미국', 0.5600796937942505)]

In [17]:
w2v_model.wv.most_similar(positive = "일본")

[('쪽바리', 0.637258768081665),
 ('일본도', 0.6027649641036987),
 ('롯게', 0.5917129516601562),
 ('좃선님', 0.5848301649093628),
 ('일본인', 0.5770894289016724),
 ('쪽발이', 0.5697472095489502),
 ('중국', 0.560553789138794),
 ('왜놈', 0.5594272613525391),
 ('넘듬', 0.5572184324264526),
 ('짓맓고', 0.5498552918434143)]

In [18]:
w2v_model.wv.most_similar(positive = "대통령")

[('대통', 0.7586009502410889),
 ('문재인', 0.6966331005096436),
 ('통령', 0.6032859086990356),
 ('영부인', 0.6013497710227966),
 ('총리', 0.5542508363723755),
 ('망녕났군', 0.5260519981384277),
 ('박성욱', 0.5248264670372009),
 ('김정숙', 0.5233049392700195),
 ('땔래야땔수없', 0.5201804041862488),
 ('부럽쥬', 0.5174517631530762)]

## fastText 모델 학습 및 불러오기

- 학습 데이터와 파라미터 동일하게 해서 이후에 word2vec과 결과 비교
- 자모 단위 모델 추가

In [12]:
print(tokens[0])
print(len(tokens))

['이국', '종', '교수', '외과', '야전', '사령관', '분', '업무', '차질', '없다', '물', '심양', '도', '주기', '분', '역활']
2393070


In [14]:
ft_config = {
    'min_count': 1,  # 등장 횟수가 1 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 100,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
    'window': 5,
    'seed': 25, #random number,
    'word_ngrams':1 # uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec.
}

In [15]:
# 모델 학습
fastText_model = fasttext.FastText(sentences=tokens, **ft_config)  

In [20]:
# 연관성 테스트
fastText_model.wv.most_similar("중국")

[('짱개', 0.7340356707572937),
 ('짱깨', 0.7271621227264404),
 ('중공', 0.7174848318099976),
 ('러시아', 0.6323301792144775),
 ('짱꼴라', 0.6283606290817261),
 ('중국인', 0.6042678356170654),
 ('중궈', 0.5787336826324463),
 ('시진핑', 0.575260579586029),
 ('일본', 0.5739111304283142),
 ('중국스립', 0.5734913349151611)]

In [16]:
# 모델 저장
fastText_model.save('dataset/embedding/fastText_0330.model')

In [48]:
# 모델 불러오기
fastText_model = fasttext.FastText.load('dataset/embedding/fastText_0330.model')

# fastText - 자모 임베딩 

<br> <b>Reference
- https://lovit.github.io/nlp/representation/2018/10/22/fasttext_subword/

In [15]:
# 자모 단위 학습
from soynlp.hangle import decompose
import re

doublespace_pattern = re.compile('\s+')

# 문장을 자모 단위로 분해
def jamo_sentence(sent):

    def transform(char):
        try:
            if char == ' ':
                return char
            cjj = decompose(char)
            if len(cjj) == 1:
                return cjj
            cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)
            return cjj_
        
        except Exception as e: # 마침표, 물음표 반환
            #print("error char: {0}   error: {1}".format(char, e))
            return char

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_

test_comment = token_data["comment"][0]
print("test_comment: ", test_comment)
jamo_sentence(test_comment)

test_comment:  이국종교수는  외과 야전사령관 이다.  그분의 업무에 차질 없도록 물심양면 으로  도와 주기만 하면 그분의 역활을 다할것이다.


'ㅇㅣ-ㄱㅜㄱㅈㅗㅇㄱㅛ-ㅅㅜ-ㄴㅡㄴ ㅇㅚ-ㄱㅘ- ㅇㅑ-ㅈㅓㄴㅅㅏ-ㄹㅕㅇㄱㅘㄴ ㅇㅣ-ㄷㅏ-. ㄱㅡ-ㅂㅜㄴㅇㅢ- ㅇㅓㅂㅁㅜ-ㅇㅔ- ㅊㅏ-ㅈㅣㄹ ㅇㅓㅄㄷㅗ-ㄹㅗㄱ ㅁㅜㄹㅅㅣㅁㅇㅑㅇㅁㅕㄴ ㅇㅡ-ㄹㅗ- ㄷㅗ-ㅇㅘ- ㅈㅜ-ㄱㅣ-ㅁㅏㄴ ㅎㅏ-ㅁㅕㄴ ㄱㅡ-ㅂㅜㄴㅇㅢ- ㅇㅕㄱㅎㅘㄹㅇㅡㄹ ㄷㅏ-ㅎㅏㄹㄱㅓㅅㅇㅣ-ㄷㅏ-.'

In [16]:
token_data["comment_jamo"] = token_data["comment"].apply(jamo_sentence)
token_data.head(5)

,news_id,comment_id,comment,comment_token,comment_jamo
0,20171201145847479,F3C2DFCFE9E947B19C8A3053DD5C821B,이국종교수는 외과 야전사령관 이다. 그분의 업무에 차질 없도록 물심양면 으로 ...,"[이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없다, 물, 심양,...",ㅇㅣ-ㄱㅜㄱㅈㅗㅇㄱㅛ-ㅅㅜ-ㄴㅡㄴ ㅇㅚ-ㄱㅘ- ㅇㅑ-ㅈㅓㄴㅅㅏ-ㄹㅕㅇㄱㅘㄴ ㅇㅣ-ㄷ...
1,20171201145847479,61A3B6AAD6124DFFB70EC4275496F8A6,"문재인 대통령님과 이국종 교수님, 그리고 귀순병사 모두 건강하시길 빕니다.","[문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하다, 비다]","ㅁㅜㄴㅈㅐ-ㅇㅣㄴ ㄷㅐ-ㅌㅗㅇㄹㅕㅇㄴㅣㅁㄱㅘ- ㅇㅣ-ㄱㅜㄱㅈㅗㅇ ㄱㅛ-ㅅㅜ-ㄴㅣㅁ,..."
2,20171201060244786,B6660EF2C7C14A2F903A08363641CF09,단 한번도 써보지 못했다ㆍ 포인트의 절반만이라도 통신비로 차감을!,"[단, 한번, 써다, 보지, 포인트, 절반, 통신비, 차감]",ㄷㅏㄴ ㅎㅏㄴㅂㅓㄴㄷㅗ- ㅆㅓ-ㅂㅗ-ㅈㅣ- ㅁㅗㅅㅎㅐㅆㄷㅏ-ㆍ ㅍㅗ-ㅇㅣㄴㅌㅡ-ㅇㅢ...
3,20171201060244786,C4F129BFB1FD4543A24A03D96F38351D,우리나라 통신업체들 참 쉽게 돈 벌어요,"[우리나라, 통신, 업체, 차다, 쉬다, 돈, 벌다]",ㅇㅜ-ㄹㅣ-ㄴㅏ-ㄹㅏ- ㅌㅗㅇㅅㅣㄴㅇㅓㅂㅊㅔ-ㄷㅡㄹ ㅊㅏㅁ ㅅㅟㅂㄱㅔ- ㄷㅗㄴ ㅂㅓ...
4,20171201060244786,1A21FDDF151E45DF808F0E5B662E8F3C,진짜 쓸데가너무적다.,"[진짜, 쓸다, 너무, 적다]",ㅈㅣㄴㅉㅏ- ㅆㅡㄹㄷㅔ-ㄱㅏ-ㄴㅓ-ㅁㅜ-ㅈㅓㄱㄷㅏ-.


In [17]:
jamo_data = token_data["comment_jamo"]
jamo_data[:5]

0    ㅇㅣ-ㄱㅜㄱㅈㅗㅇㄱㅛ-ㅅㅜ-ㄴㅡㄴ ㅇㅚ-ㄱㅘ- ㅇㅑ-ㅈㅓㄴㅅㅏ-ㄹㅕㅇㄱㅘㄴ ㅇㅣ-ㄷ...
1    ㅁㅜㄴㅈㅐ-ㅇㅣㄴ ㄷㅐ-ㅌㅗㅇㄹㅕㅇㄴㅣㅁㄱㅘ- ㅇㅣ-ㄱㅜㄱㅈㅗㅇ ㄱㅛ-ㅅㅜ-ㄴㅣㅁ,...
2    ㄷㅏㄴ ㅎㅏㄴㅂㅓㄴㄷㅗ- ㅆㅓ-ㅂㅗ-ㅈㅣ- ㅁㅗㅅㅎㅐㅆㄷㅏ-ㆍ ㅍㅗ-ㅇㅣㄴㅌㅡ-ㅇㅢ...
3    ㅇㅜ-ㄹㅣ-ㄴㅏ-ㄹㅏ- ㅌㅗㅇㅅㅣㄴㅇㅓㅂㅊㅔ-ㄷㅡㄹ ㅊㅏㅁ ㅅㅟㅂㄱㅔ- ㄷㅗㄴ ㅂㅓ...
4                        ㅈㅣㄴㅉㅏ- ㅆㅡㄹㄷㅔ-ㄱㅏ-ㄴㅓ-ㅁㅜ-ㅈㅓㄱㄷㅏ-.
Name: comment_jamo, dtype: object

In [19]:
# 모델 학습
fastText_model = fasttext.FastText(sentences=jamo_data, **ft_config)  

In [23]:
# 연관성 테스트
fastText_model.wv.most_similar(jamo_sentence("중국"))

[('*', 0.17194263637065887),
 ('o', 0.1222517192363739),
 ('◀', 0.1213143840432167),
 ('l', 0.09772109240293503),
 ('】', 0.09191815555095673),
 ('H', 0.08432599157094955),
 ('秦', 0.08144447952508926),
 ('♨', 0.08074280619621277),
 ('a', 0.080184206366539),
 ('§', 0.07871407270431519)]

In [24]:
# 모델 저장
fastText_model.save(token_data_PATH + "fastText_jamo.model")

In [ ]:
model = fasttext.train_unsupervised("train.txt", model='skipgram', lr=0.05, dim=100, ws=5, epoch=5)
model.save_model("model_file.bin")

In [49]:
jamo_data

0          ㅇㅣ-ㄱㅜㄱㅈㅗㅇㄱㅛ-ㅅㅜ-ㄴㅡㄴ ㅇㅚ-ㄱㅘ- ㅇㅑ-ㅈㅓㄴㅅㅏ-ㄹㅕㅇㄱㅘㄴ ㅇㅣ-ㄷ...
1          ㅁㅜㄴㅈㅐ-ㅇㅣㄴ ㄷㅐ-ㅌㅗㅇㄹㅕㅇㄴㅣㅁㄱㅘ- ㅇㅣ-ㄱㅜㄱㅈㅗㅇ ㄱㅛ-ㅅㅜ-ㄴㅣㅁ,...
2          ㄷㅏㄴ ㅎㅏㄴㅂㅓㄴㄷㅗ- ㅆㅓ-ㅂㅗ-ㅈㅣ- ㅁㅗㅅㅎㅐㅆㄷㅏ-ㆍ ㅍㅗ-ㅇㅣㄴㅌㅡ-ㅇㅢ...
3          ㅇㅜ-ㄹㅣ-ㄴㅏ-ㄹㅏ- ㅌㅗㅇㅅㅣㄴㅇㅓㅂㅊㅔ-ㄷㅡㄹ ㅊㅏㅁ ㅅㅟㅂㄱㅔ- ㄷㅗㄴ ㅂㅓ...
4                              ㅈㅣㄴㅉㅏ- ㅆㅡㄹㄷㅔ-ㄱㅏ-ㄴㅓ-ㅁㅜ-ㅈㅓㄱㄷㅏ-.
                                 ...                        
2393065    ㅇㅣㄴㅊㅓㄴㅇㅔ- ㅈㅏ-ㅇㅠ- ㅁㅝ-ㅅㅣ-ㄱㅣ- ㅅㅗ-ㅅㅗㄱ ㄱㅜ-ㅊㅓㅇㅈㅏㅇㅇㅣ-...
2393066          ㅁㅐㅇㅂㅏ-ㄱㅣ-ㄷㅏㄻㅇㅡㄴㄱㅓㅅㅂㅗ-ㄴㅣ- ㅈㅏ-ㅇㅠ-ㄷㅏㅇㅇㅣㄹㄷㅡㅅ...
2393067    ㅁㅓㅇㅂㅏㄱㄱㅡ-ㄴㅔ- ㅈㅓㅇㄱㅝㄴ ㅆㅏ- ㄸㅗㅇㅊㅣ-ㅇㅜ-ㄴㅡㄴ ㅁㅜㄴㅈㅓㅇㅂㅜ-...
2393068    ㅇㅣ-ㅈㅔ- ㅁㅕㅇㅂㅏㄱㅇㅣ-ㅈㅗㅁ ㅇㅣㅈㅇㅡㄹㄸㅐ-ㄷㅗ- ㅇㅏㄴㄷㅙㅆㄴㅏ- ㄱㅡ-...
2393069    Cㄱㅜ-ㅊㅓㅇㅈㅏㅇㅇㅡㄹ ㄷㅏㅇㅈㅏㅇ ㄱㅜ-ㅅㅗㄱ ㅅㅜ-ㅅㅏ-ㅎㅏ-ㅁㅕㄴ ㄷㅚㄹㄷㅡ...
Name: comment_jamo, Length: 2393070, dtype: object

In [48]:
# utf-8 인코딩 기반의 텍스트 파일로 저장
jamo_data.to_csv('D:\\WorkSpace\\뉴스 데이터\\200401_embedding_data\\jamo_data.txt', 
                 index=False, header=None, sep="\n", encoding = 'utf-8')

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
f = open('D:\\WorkSpace\\뉴스 데이터\\200401_embedding_data\\jamo_data.txt', 'w', encoding = 'utf-8')

for i in token_data["comment_jamo"]:
    f.write(i)
    f.write('\n')

f.close()